# Modified Gale Shapley Algorithm

In [1]:
import pandas as pd

In [2]:
syndata = pd.read_csv('synData9.csv')
realdata = pd.read_csv('kag_risk_factors_cervical_cancer.csv')

In [3]:
syndata.iloc[0] #Use this to access a row
syndata.loc[0, :] #Use this to access a row
x = syndata.loc[:,"Age"] #Use this to access a column

In [6]:
syndata.columns[1]

'Number of sexual partners'

Now we work on creating a compare function which is being designed to comapre all the rows of one dataset to one specific row of another dataset

In [25]:
def compare(data1, data2, row_comp):
    """
    This is a comparsion function  
    data1: First dataset for comparsion, 1 set row
    data2: Second dataset for comparsion
    row_comp: The row we want compared
    """
    for i in range(len(data2)):
        print('Real data: ' + str(data1.loc[row_comp, 'Age'])+ '. Fake data: ' +str(data2.loc[i, 'Age']))
        #similiar(syndata)

In [19]:
def similiar(data1):
    """
    Computes how similiar the two datasets are 
    """
    for i in range(len(data1.columns)):
        print(data1.loc[:,data1.columns[i]])

In [ ]:
similiar(syndata)

In [26]:
compare(realdata, syndata, 1)

Real data: 15. Fake data: 12
Real data: 15. Fake data: 19
Real data: 15. Fake data: 21
Real data: 15. Fake data: 29
Real data: 15. Fake data: 24
Real data: 15. Fake data: 35
Real data: 15. Fake data: 26
Real data: 15. Fake data: 38
Real data: 15. Fake data: 19
Real data: 15. Fake data: 13
Real data: 15. Fake data: 21
Real data: 15. Fake data: 11
Real data: 15. Fake data: 17
Real data: 15. Fake data: 25
Real data: 15. Fake data: 21
Real data: 15. Fake data: 14
Real data: 15. Fake data: 48
Real data: 15. Fake data: 29
Real data: 15. Fake data: 30
Real data: 15. Fake data: 30
Real data: 15. Fake data: 17
Real data: 15. Fake data: 33
Real data: 15. Fake data: 36
Real data: 15. Fake data: 21
Real data: 15. Fake data: 32
Real data: 15. Fake data: 21
Real data: 15. Fake data: 21
Real data: 15. Fake data: 27
Real data: 15. Fake data: 25
Real data: 15. Fake data: 27
Real data: 15. Fake data: 33
Real data: 15. Fake data: 20
Real data: 15. Fake data: 45
Real data: 15. Fake data: 31
Real data: 15.